In [255]:
#imports 
import requests
from pprint import pprint
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os
import warnings
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

import difflib 

#database connection
import psycopg2 as psql 
from dotenv import load_dotenv
import os
import base64
import pprint

from statistics import mean
import datetime

In [284]:
#get secrets from .env
load_dotenv()

api_key = os.getenv('API_KEY')
NASA_api_key = os.getenv('NASA_API_KEY')
METEOSTAT_api_key = os.getenv('METEOSTAT_API_KEY')

### Connect to API

### Nasa 

#### Different api stuff to add


Climate
* weather - open-meteo
* NASA natural disasters - https://eonet.gsfc.nasa.gov/api/v2.1/events ?days=20


Space Objects
* NEO - https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key=DEMO_KEY
* JPL’s SSD (Solar System Dynamics) and CNEOS (Center for Near-Earth Object Studies) - https://ssd-api.jpl.nasa.gov/


Space Pictures
* picture of the day - https://api.nasa.gov/planetary/apod?api_key=DEMO_KEY&count=5
* mars photo api - https://api.nasa.gov/mars-photos/
* earth picture - https://epic.gsfc.nasa.gov/


### Climate

#### Weather

In [288]:
df = pd.read_csv('Data/weather.csv')
df

In [289]:
df

,country,capital,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,Afghanistan,Kabul,1966-03-02,winter,7.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Kabul,1966-03-28,spring,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Kabul,1966-05-02,spring,18.8,NaN,22.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Kabul,1966-05-04,spring,19.7,NaN,27.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Kabul,1966-05-18,spring,24.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5002446,Zimbabwe,Harare,2023-08-15,winter,20.1,11.0,28.1,0.0,NaN,120.0,8.5,NaN,1018.5,NaN
5002447,Zimbabwe,Harare,2023-08-16,winter,18.6,12.4,25.6,0.0,NaN,116.0,17.4,NaN,1021.2,NaN
5002448,Zimbabwe,Harare,2023-08-17,winter,17.5,10.7,24.7,0.0,NaN,94.0,14.3,NaN,1020.9,NaN
5002449,Zimbabwe,Harare,2023-08-18,winter,18.9,10.1,27.2,0.0,NaN,56.0,9.1,NaN,1018.2,NaN


##### Set up DB Data

In [280]:
# populate db with past data first
#df = pd.read_parquet('Data/daily_weather.parquet')
df = pd.read_csv('Data/weather.csv')

#remove columns
weather_df = df[['country', 'capital', 'date', 'avg_temp_c', 'precipitation_mm','avg_wind_speed_kmh']]

#add long and lats for each city
countries_df = pd.read_csv('Data/country-capital-lat-long-population.csv')
countries_df = countries_df[['Country','Latitude', 'Longitude']]
countries_df.rename(columns={'Country': 'country', 'Latitude' : 'latitude', 'Longitude' : 'longitude'}, inplace=True)

#add mssing lat long manually
data = {
    'country': ['Bolivia','British Indian Ocean Territory', 'Brunei', 'Cape Verde',
       'Christmas Island', 'Cocos (Keeling) Islands', 'Czech Republic',
       'East Timor', 'Falkland Islands', 'Faroe Islands',
       'Federated States of Micronesia',
       'French Southern and Antarctic Lands', 'Guernsey', 'Hong Kong',
       'Iran', 'Ivory Coast', 'Jersey', 'Laos', 'Macedonia', 'Moldova',
       'Norfolk Island', 'North Korea', 'Pitcairn Islands',
       'Republic of the Congo', 'Russia', 'South Georgia', 'South Korea',
       'Svalbard and Jan Mayen', 'Syria', 'São Tomé and Príncipe',
       'Taiwan', 'Tanzania', 'The Bahamas', 'The Gambia', 'United States',
       'Venezuela', 'Vietnam', 'Wallis and Futuna'],
    'latitude': [-16.5000, -7.3195, 4.8903, 14.9167, -10.4167, -12.1167, 50.0833,
       -8.5594, -51.7000, 62.0167, 6.9167, -49.3500, 49.4500, 22.2783,
       35.6944, 6.8167, 49.1833, 17.9667, 42.0000, 47.0056, -29.0569,
       39.0194, -25.0667, -4.2592, 55.7558, -54.2833, 37.5665, 78.2333,
       33.5000, 0.3333, 25.0330, -6.8000, 25.0833, 13.4531, 38.8951,
       10.4806, 21.0285, -13.2833],
    'longitude': [-68.1500, 72.4229, 114.9403, -23.5167, 105.7167, 96.9000, 14.4167,
       125.5795, -57.8500, -6.7667, 158.1500, 70.2167, -2.5333, 114.1747,
       51.4215, -5.2667, -2.1000, 102.6000, 21.4333, 28.8575, 167.9597,
       125.7547, -130.1000, 15.2847, 37.6173, -36.5000, 126.9780, 15.6333,
       36.3000, 6.7333, 121.5654, 39.2833, -77.3500, -16.5775, -77.0364,
       -66.9036, 105.8542, -176.1667]
}
data_df = pd.DataFrame(data)
countries_df = pd.concat([countries_df, data_df], ignore_index=True)

merged_df = weather_df.merge(countries_df, on='country', how='left')
merged_df.tail()

,country,capital,date,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,latitude,longitude
5002446,Zimbabwe,Harare,2023-08-15,20.1,0.0,8.5,-17.8294,31.0539
5002447,Zimbabwe,Harare,2023-08-16,18.6,0.0,17.4,-17.8294,31.0539
5002448,Zimbabwe,Harare,2023-08-17,17.5,0.0,14.3,-17.8294,31.0539
5002449,Zimbabwe,Harare,2023-08-18,18.9,0.0,9.1,-17.8294,31.0539
5002450,Zimbabwe,Harare,2023-08-19,21.1,0.0,11.3,-17.8294,31.0539


In [285]:
#Fill rest of dates for weather from API +2023-08-20
def process_json_response(jsonRes, latitude, longitude):
    data_list = []
    
    for day_data in jsonRes['data']:
        data = {
            'date': day_data['date'],
            'avg_temp_c': day_data['tavg'],
            'precipitation_mm': day_data['prcp'],
            'avg_wind_speed_kmh': day_data['wspd'],
            'latitude': latitude,
            'longitude': longitude
        }
        data_list.append(data)
    
    return pd.DataFrame(data_list)


def get_weather_data(lat, lon):
    url = "https://meteostat.p.rapidapi.com/point/daily"
    querystring = {"lat":str(lat),"lon":str(lon),"start":"2023-08-20","end":"2024-06-26"}
    headers = {
        "x-rapidapi-key": METEOSTAT_api_key,
        "x-rapidapi-host": "meteostat.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

# Get unique pairs of latitude and longitude
coord_pairs = merged_df[['latitude', 'longitude']].drop_duplicates().values

# List to store DataFrames for each coordinate pair
df_list = []

# Loop through each coordinate pair
for lat, lon in coord_pairs:
    print(f"Fetching data for coordinates: {lat}, {lon}")
    
    try:
        # Make API call
        jsonRes = get_weather_data(lat, lon)
        
        # Process the response
        df = process_json_response(jsonRes, lat, lon)
        
        # Add to the list of DataFrames
        df_list.append(df)
        
    except Exception as e:
        print(f"Error fetching data for coordinates {lat}, {lon}: {str(e)}")

# Combine all DataFrames
combined_df = pd.concat(df_list, ignore_index=True)
combined_df = pd.DataFrame(combined_df)
# Display the resulting DataFrame
print(combined_df)

Fetching data for coordinates: 34.5289, 69.1725
Fetching data for coordinates: 41.3275, 19.8189
Fetching data for coordinates: 36.7525, 3.042
Fetching data for coordinates: -14.2781, -170.7025
Fetching data for coordinates: -8.8368, 13.2343
Fetching data for coordinates: 18.217, -63.0578
Fetching data for coordinates: 17.1172, -61.8457
Fetching data for coordinates: -34.6051, -58.4004
Fetching data for coordinates: 40.182, 44.5146
Fetching data for coordinates: 12.524, -70.027
Fetching data for coordinates: -35.2835, 149.1281
Fetching data for coordinates: 48.2064, 16.3707
Fetching data for coordinates: 40.3777, 49.892
Fetching data for coordinates: 26.2154, 50.5832
Fetching data for coordinates: 23.7104, 90.4074
Fetching data for coordinates: 13.1, -59.6167
Fetching data for coordinates: 53.9, 27.5667
Fetching data for coordinates: 50.8467, 4.3499
Fetching data for coordinates: 17.25, -88.7667
Fetching data for coordinates: 6.3654, 2.4183
Fetching data for coordinates: 32.2915, -64.77

In [286]:
#add countries and capitals to new weather data 
coord_pairs = merged_df[['country','capital','latitude', 'longitude']].drop_duplicates().values

# Convert coord_pairs to a DataFrame
coord_df = pd.DataFrame(coord_pairs, columns=['country', 'capital', 'latitude', 'longitude'])
df_with_countries = combined_df.merge(coord_df, on=['latitude', 'longitude'], how='left')

#combine both weather datasets
full_weather_df = pd.concat([merged_df,df_with_countries],ignore_index=True)
#save weather to csv
full_weather_df.to_csv('Data/historical_weather_data.csv', index=False)
full_weather_df.tail()

,country,capital,date,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,latitude,longitude
5071710,Zimbabwe,Harare,2024-06-22,13.6,0.0,11.1,-17.8294,31.0539
5071711,Zimbabwe,Harare,2024-06-23,13.7,0.0,14.1,-17.8294,31.0539
5071712,Zimbabwe,Harare,2024-06-24,13.5,0.0,14.8,-17.8294,31.0539
5071713,Zimbabwe,Harare,2024-06-25,13.6,0.0,13.2,-17.8294,31.0539
5071714,Zimbabwe,Harare,2024-06-26,13.0,0.0,13.1,-17.8294,31.0539


##### Set up daily api calls

In [238]:
## get daily weather data for capital cities
## additional data for each place: Air quality, Water Quality, Temperature 
## set up api daily connection

# 230 calls daily  for weather 
#url = f'https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41'

#url = f'https://api.openweathermap.org/data/2.5/weather?q={cityname}&appid={API key}'

latitudes = merged_df['latitude'].unique()
longitudes = merged_df['longitude'].unique()

# Convert float values to strings and join them with commas
lat_string = ",".join(map(str, latitudes))
long_string = ",".join(map(str, longitudes))

#precipitation_mm	avg_wind_speed_kmh	avg_sea_level_pres_hpa

#https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41
url = f'https://api.open-meteo.com/v1/forecast?latitude={lat_string}&longitude={long_string}&start_date=2024-06-27&end_date=2024-06-27&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max'
#url = 'https://api.open-meteo.com/v1/forecast?latitude=52.52,51.5085&longitude=13.41,-0.1257&hourly=temperature_2m'

#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
jsonRes = response.json()
#print(jsonRes) 


In [239]:
rows = []

for location in jsonRes:
    lat = location['latitude']
    lon = location['longitude']
    times = location['daily']['time']
    max_temps = location['daily']['temperature_2m_max']
    min_temps = location['daily']['temperature_2m_min']
    
    for date, max_temp, min_temp in zip(times, max_temps, min_temps):
        # Calculate average temperature if both max and min are available
        if max_temp is not None and min_temp is not None:
            avg_temp = mean([max_temp, min_temp])          
        else:
            avg_temp = None
        
        rows.append({
            'date': date,
            'avg_temp_c': avg_temp,
            'latitude': lat,
            'longitude': lon
        })

testerweatherapi = pd.DataFrame(rows)
testerweatherapi

,date,avg_temp_c,latitude,longitude
0,2024-06-27,26.20,34.6250,69.125000
1,2024-06-27,22.20,41.3125,19.812500
2,2024-06-27,24.70,36.7500,3.062500
3,2024-06-27,26.60,-14.2500,-170.750000
4,2024-06-27,23.65,-8.8750,13.250000
...,...,...,...,...
225,2024-06-27,27.90,-13.2500,-176.125000
226,2024-06-27,22.15,27.2000,-13.199999
227,2024-06-27,24.40,15.3750,44.250000
228,2024-06-27,15.80,-15.3750,28.250000


#### Earthquakes

##### Set up DB Data

In [275]:
# populate db with historical data first
earthquake_data=pd.read_csv('Data/Significant_Earthquakes.csv') #+5.0 mag
#manipulate to only have capital cites
earthquake_data = earthquake_data[['time', 'latitude', 'longitude', 'mag', 'magType', 'place','type']]
earthquake_data = earthquake_data[earthquake_data['type'] == 'earthquake']
earthquake_data = earthquake_data.drop(columns=['type'])

#change time format
earthquake_data['time'] = pd.to_datetime(earthquake_data['time']) 

earthquake_data


,time,latitude,longitude,mag,magType,place
0,1900-10-09 12:25:00+00:00,57.09,-153.48,7.86,mw,"16 km SW of Old Harbor, Alaska"
1,1901-03-03 07:45:00+00:00,36.00,-120.50,6.40,ms,"12 km NNW of Parkfield, California"
2,1901-07-26 22:20:00+00:00,40.80,-115.70,5.00,fa,"6 km SE of Elko, Nevada"
3,1901-12-30 22:34:00+00:00,52.00,-160.00,7.00,ms,south of Alaska
4,1902-01-01 05:20:30+00:00,52.38,-167.45,7.00,ms,"113 km ESE of Nikolski, Alaska"


In [279]:
## get historical data of earthquakes
## set up api daily connection

#get earthquakes from 2024-05-25+

#https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02
#url = f'https://earthquake.usgs.gov/fdsnws/event/1/[METHOD[?PARAMETERS]]'
url = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-25&endtime=2024-06-27&minmagnitude=5'

#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
jsonRes = response.json()

#collect data
extra_earthquake_list = []
for earthquake in jsonRes['features']:
        data = {
            'time': earthquake['properties']['time'],
            'latitude': earthquake['geometry']['coordinates'][1],
            'longitude': earthquake['geometry']['coordinates'][0],
            'mag': earthquake['properties']['mag'],
            'magType': earthquake['properties']['magType'],
            'place': earthquake['properties']['place']
        }  
        extra_earthquake_list.append(data)
    
extra_earthquake_df = pd.DataFrame(extra_earthquake_list)
extra_earthquake_df['time'] = pd.to_datetime(extra_earthquake_df['time'], unit='ms',origin='unix', utc=True) 
extra_earthquake_df[:] = extra_earthquake_df[::-1]
extra_earthquake_df

#append to all earthquakes dataset
full_earthquake_df = pd.concat([earthquake_data,extra_earthquake_df],ignore_index=True)
full_earthquake_df.to_csv('Data/historical_earthquake_data.csv', index=False)
full_earthquake_df


,time,latitude,longitude,mag,magType,place
0,1900-10-09 12:25:00+00:00,57.0900,-153.4800,7.86,mw,"16 km SW of Old Harbor, Alaska"
1,1901-03-03 07:45:00+00:00,36.0000,-120.5000,6.40,ms,"12 km NNW of Parkfield, California"
2,1901-07-26 22:20:00+00:00,40.8000,-115.7000,5.00,fa,"6 km SE of Elko, Nevada"
3,1901-12-30 22:34:00+00:00,52.0000,-160.0000,7.00,ms,south of Alaska
4,1902-01-01 05:20:30+00:00,52.3800,-167.4500,7.00,ms,"113 km ESE of Nikolski, Alaska"
...,...,...,...,...,...,...
102161,2024-06-25 06:45:33.808000+00:00,-20.9050,-173.8305,5.10,mb,"126 km ENE of ‘Ohonua, Tonga"
102162,2024-06-25 08:20:46.696000+00:00,-54.2645,-56.9279,5.00,mb,Falkland Islands region
102163,2024-06-25 08:30:59.282000+00:00,33.4887,141.2838,5.40,mww,"204 km SSE of Katsuura, Japan"
102164,2024-06-26 01:24:13.670000+00:00,8.0754,90.4097,5.60,mww,"Nicobar Islands, India region"


##### Set up daily api calls

In [ ]:
todaysDate = '2024-05-25'
url = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={todaysDate}&endtime={todaysDate}&minmagnitude=5'

#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
jsonRes = response.json()

#### Air Quality

In [ ]:
url = f'https://aqicn.org/api'

#### Natural disasters 

In [75]:
#url = f'https://eonet.gsfc.nasa.gov/api/v2.1/events?days=20'
url = f'https://eonet.gsfc.nasa.gov/api/v2.1/events'

#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
jsonRes = response.json()
print(jsonRes);


{'title': 'EONET Events', 'description': 'Natural events from EONET.', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events', 'events': [{'id': 'EONET_8212', 'title': '0289 NE DARLENE 3 Wildfire, Deschutes, Oregon', 'description': 'One Mile South Southeast of the city of La Pine, Oregon', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events/EONET_8212', 'categories': [{'id': 8, 'title': 'Wildfires'}], 'sources': [{'id': 'IRWIN', 'url': 'https://irwin.doi.gov/observer/'}], 'geometries': [{'date': '2024-06-25T16:15:00Z', 'type': 'Point', 'coordinates': [-121.413517, 44.267745]}]}, {'id': 'EONET_8225', 'title': 'APACHE Wildfire, Butte, California', 'description': 'Palermo, South of Oroville', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events/EONET_8225', 'categories': [{'id': 8, 'title': 'Wildfires'}], 'sources': [{'id': 'IRWIN', 'url': 'https://irwin.doi.gov/observer/'}], 'geometries': [{'date': '2024-06-24T22:21:00Z', 'type': 'Point', 'coordinates': [-121.482631, 39.418598]}]}, {'id'

### Space

#### Nasa Picture of the Day

In [12]:
#url = f'https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key={NASA_api_key}'
url = f'https://api.nasa.gov/planetary/apod?api_key={NASA_api_key}'

#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
jsonRes = response.json()
print(jsonRes) 


{'copyright': '\nJeff Dai\n(TWAN); \nMusic \n(License): \nSuite bergamasque by \nClaude Debussy\n', 'date': '2024-06-26', 'explanation': "What's happening in the sky this unusual night? Most striking in the featured 4.5-hour 360-degree panoramic video, perhaps, is the pink and purple aurora. That's because this night, encompassing May 11, was famous for its auroral skies around the world. As the night progresses, auroral bands shimmer, the central band of our Milky Way Galaxy rises, and stars shift as the Earth rotates beneath them.  Captured here simultaneously is a rare red band running above the aurora: a SAR arc, seen to change only slightly. The flashing below the horizon is caused by passing cars, while the moving spots in the sky are satellites and airplanes.  The featured video was captured from Xinjiang, China with four separate cameras.", 'media_type': 'video', 'service_version': 'v1', 'title': 'Timelapse: Aurora, SAR, and the Milky Way', 'url': 'https://www.youtube.com/embed

#### Asteroids nearby

In [134]:
#url = f'https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key={NASA_api_key}'
#url = f'https://api.nasa.gov/neo/rest/v1/neo/browse?api_key={NASA_api_key}'
#current date astroid is closest to earth
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-06-26&end_date=2024-06-26&api_key={NASA_api_key}'
#headers = { 'X-Auth-Token': api_key }

response = requests.get(url)

#print(response) #response code 200: Good 400: Bad
neoJSON = response.json()
#print(neoJSON)

In [135]:
miss_miles = [x['close_approach_data'][0]['miss_distance']['miles'] for x in neoJSON["near_earth_objects"]['2024-06-26']]
diameter_miles = [x['estimated_diameter']['miles']['estimated_diameter_max'] for x in neoJSON["near_earth_objects"]['2024-06-26']]
is_hazardous = [x['is_potentially_hazardous_asteroid'] for x in neoJSON["near_earth_objects"]['2024-06-26']]
print(miss_miles)
print(diameter_miles)
print(is_hazardous)

['30082164.1062733288', '41823340.5999610626', '28544496.757910679', '44150778.4165702704', '41308947.096381527', '8913021.305752321', '10215776.5137473234']
[0.3085964738, 0.0771597762, 0.0029880713, 0.0139120445, 0.037272709, 0.0100783853, 0.1774152548]
[False, False, False, False, False, False, False]


### Connect to DB

In [ ]:
#get secrets from .env
load_dotenv()

username = os.getenv('sql_user')
password = os.getenv('sql_pass')
host = os.getenv('host')

conn = psql.connect(database = 'pagila', 
                    user = username, 
                    host = host, 
                    password = password, 
                    port = 5432)

cur = conn.cursor()
SQL = """
CREATE TABLE IF NOT EXISTS student.mytable (

  my_id serial primary key,
  my_str varchar(50)
)
"""


cur.execute(SQL)
conn.commit()
